# Versión tres del modelo (Regresión logistica)
Voy a probar ahora el otro modelo, pero con los mismos datos que en la v2.

Hay que solucionar todavía lo del pd.merge()

Update1 : Va mejor que el RandomForest de la versión v2 (para las mismas variables). Precisión del modelo: 0.4558315391790497

Update2: Le he metido una variable más que es el equipo home. No cambia casi nada. Puede que sea porque no le he metido muchos datos para el train. Precisión del modelo: 0.45270325763664415

In [22]:
# Importo los paquetes
import pandas as pd
import numpy as np
import sqlite3
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression # el modelo
from sklearn.metrics import accuracy_score, classification_report # para ver cómo funciona
from sklearn.preprocessing import LabelEncoder # poner los datos tipo int



In [23]:
# Importo los dataframes
with sqlite3.connect("../laliga.sqlite") as conn:
        df = pd.read_sql("SELECT * FROM Matches", conn)
conn.close() 

In [24]:
# Importo los dataframes
with sqlite3.connect("clasification.sqlite") as conn:
        df_teams = pd.read_sql("SELECT * FROM clasification", conn)
conn.close()  

In [25]:
# Si pongo directamente df_games =con(...) me da error en la siguiente celda
df_games=df  

In [26]:
# Le creo a df_games la columna de result (home, visitor, tie) que va a ser mi variable Y en el modelo
df_games["result"] = None
#estas dos columnas nos harían falta
df_games["goals_home"] = df_games["score"].str.split(":").str[0]
df_games["goals_visitor"] = df_games["score"].str.split(":").str[1]

df_games.loc[df["goals_home"] > df_games["goals_visitor"], "result"] = 'home'
df_games.loc[df["goals_home"] == df_games["goals_visitor"], "result"] = 'tie'
df_games.loc[df["goals_home"] < df_games["goals_visitor"], "result"] = 'visitor'

In [27]:
df_games.drop(["goals_home", "goals_visitor", "time", "date", "score"], axis=1, inplace=True)  
df_games

,season,division,matchday,home_team,away_team,result
0,1928-1929,1,1,Arenas Club,Athletic Madrid,visitor
1,1928-1929,1,1,Espanyol,Real Unión,home
2,1928-1929,1,1,Real Madrid,Catalunya,home
3,1928-1929,1,1,Donostia,Athletic,tie
4,1928-1929,1,1,Racing,Barcelona,visitor
...,...,...,...,...,...,...
48775,2021-2022,2,42,Real Oviedo,UD Ibiza,None
48776,2021-2022,2,42,Real Sociedad B,Real Zaragoza,None
48777,2021-2022,2,42,Sporting Gijón,UD Las Palmas,None
48778,2021-2022,2,42,CD Tenerife,FC Cartagena,None


# Empieza el modelo (RandomForestClassifier)
Nuestro modelo/problema es de tipo clasificación. La predicción nos tiene que dar uno de los tres valores, NO es una variable continua.

Ya tenemos los datos. Ahora hay que definir nuestras variables. X son los datos que vamos a tener en cuenta, e Y es la "variable etiqueta", es decir, la que tenemos que predecir. La etiqueta tiene que ser de tipo int, empezamos con eso.

In [28]:
division = 1
season = "2019-2020"
matchday = 3
season_train = "2000:2001"


In [29]:
# Luego, aplicar las operaciones de cadena de caracteres
df_teams_train = df_teams

# Asegurarte de que la columna "season" del DataFrame sea de tipo cadena de caracteres
df_teams_train["season"] = df_teams_train["season"].astype(str)

df_teams_train = df_teams_train[(df_teams_train["season"].str.split("-").str[0] >= season_train.split(":")[0]) &
                    (df_teams_train["season"].str.split("-").str[1] <= season_train.split(":")[1])]
df_teams_train


,season,division,rank,team,matchday,GF,GA,GD,W,L,T,Pts
35364,2000-2001,1,1.0,Rayo Vallecano,1,5.0,1.0,4.0,1.0,0.0,0.0,3.0
35365,2000-2001,1,2.0,Alavés,1,3.0,0.0,3.0,1.0,0.0,0.0,3.0
35366,2000-2001,1,3.0,Celta de Vigo,1,2.0,0.0,2.0,1.0,0.0,0.0,3.0
35367,2000-2001,1,4.0,Dep. La Coruña,1,2.0,0.0,2.0,1.0,0.0,0.0,3.0
35368,2000-2001,1,5.0,Barcelona,1,2.0,1.0,1.0,1.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
37043,2000-2001,2,18.0,Elche CF,42,45.0,56.0,-11.0,10.0,16.0,16.0,46.0
37044,2000-2001,2,19.0,SD Compostela,42,41.0,65.0,-24.0,11.0,19.0,12.0,45.0
37045,2000-2001,2,20.0,Uni. Las Palmas,42,34.0,63.0,-29.0,8.0,19.0,15.0,39.0
37046,2000-2001,2,21.0,Getafe,42,42.0,65.0,-23.0,8.0,23.0,11.0,35.0


In [30]:
df_games_train = df_games
# Asegurarte de que la columna "season" del DataFrame sea de tipo cadena de caracteres

df_games_train["season"] = df_games_train["season"].astype(str)

# Luego, aplicar las operaciones de cadena de caracteres
df_games_train = df_games_train[(df_games_train["season"].str.split("-").str[0] >= season_train.split(":")[0]) &
                    (df_games_train["season"].str.split("-").str[1] <= season_train.split(":")[1])]
df_games_train


,season,division,matchday,home_team,away_team,result
17838,2000-2001,1,1,Real Sociedad,Racing,tie
17839,2000-2001,1,1,Real Zaragoza,Espanyol,visitor
17840,2000-2001,1,1,Barcelona,Málaga CF,home
17841,2000-2001,1,1,Dep. La Coruña,Athletic,home
17842,2000-2001,1,1,Real Madrid,Valencia,home
...,...,...,...,...,...,...
38693,2000-2001,2,42,Albacete,Uni. Las Palmas,visitor
38694,2000-2001,2,42,Elche CF,SD Compostela,tie
38695,2000-2001,2,42,Sporting Gijón,Córdoba CF,tie
38696,2000-2001,2,42,Real Jaén CF,Real Betis,visitor


In [31]:
# Hacemos el merge del dataframe para train
df_train = pd.merge(df_games_train, df_teams_train, left_on='home_team', right_on='team', how='left', suffixes=("_home", ""))
df_train = pd.merge(df_train, df_teams_train, left_on='away_team', right_on='team', how='left', suffixes=("_away",""))
df_train

,season_home,division_home,matchday_home,home_team,away_team,result,season_away,division_away,rank_away,team_away,...,rank,team,matchday,GF,GA,GD,W,L,T,Pts
0,2000-2001,1,1,Real Sociedad,Racing,tie,2000-2001,1,10.0,Real Sociedad,...,9.0,Racing,1,2.0,2.0,0.0,0.0,0.0,1.0,1.0
1,2000-2001,1,1,Real Sociedad,Racing,tie,2000-2001,1,10.0,Real Sociedad,...,18.0,Racing,2,2.0,5.0,-3.0,0.0,1.0,1.0,1.0
2,2000-2001,1,1,Real Sociedad,Racing,tie,2000-2001,1,10.0,Real Sociedad,...,19.0,Racing,3,3.0,8.0,-5.0,0.0,2.0,1.0,1.0
3,2000-2001,1,1,Real Sociedad,Racing,tie,2000-2001,1,10.0,Real Sociedad,...,18.0,Racing,4,3.0,8.0,-5.0,0.0,2.0,2.0,2.0
4,2000-2001,1,1,Real Sociedad,Racing,tie,2000-2001,1,10.0,Real Sociedad,...,20.0,Racing,5,4.0,10.0,-6.0,0.0,3.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363683,2000-2001,2,42,Getafe,Atlético Madrid,visitor,2000-2001,2,21.0,Getafe,...,5.0,Atlético Madrid,38,55.0,39.0,16.0,17.0,10.0,11.0,62.0
1363684,2000-2001,2,42,Getafe,Atlético Madrid,visitor,2000-2001,2,21.0,Getafe,...,4.0,Atlético Madrid,39,56.0,39.0,17.0,18.0,10.0,11.0,65.0
1363685,2000-2001,2,42,Getafe,Atlético Madrid,visitor,2000-2001,2,21.0,Getafe,...,4.0,Atlético Madrid,40,57.0,39.0,18.0,19.0,10.0,11.0,68.0
1363686,2000-2001,2,42,Getafe,Atlético Madrid,visitor,2000-2001,2,21.0,Getafe,...,4.0,Atlético Madrid,41,58.0,39.0,19.0,20.0,10.0,11.0,71.0


In [39]:
# Supongamos que tienes un DataFrame df con una columna 'etiqueta' de tipo 'object'
le = LabelEncoder()
df_train['prediction'] = le.fit_transform(df_train['result'])
df_train["home"] = le.fit_transform(df_train["home_team"])

In [40]:
# Selecciona las características (X_train) y etiquetas (y_train)
X_train = df_train[['home', 'rank', 'GD', "W", "Pts"]]  
y_train = df_train['prediction']  

In [41]:
df_teams_test = df_teams[df_teams["season"] == season]
df_games_test = df_games[df_games["season"] == season]

In [42]:
# Hacemos el merge del dataframe para hacer el test
df_test = pd.merge(df_games_test, df_teams_test, left_on='home_team', right_on='team', how='left', suffixes=("_home", ""))
df_test = pd.merge(df_test, df_teams_test, left_on='away_team', right_on='team', how='left', suffixes=("_away",""))
df_test

,season_home,division_home,matchday_home,home_team,away_team,result,season_away,division_away,rank_away,team_away,...,rank,team,matchday,GF,GA,GD,W,L,T,Pts
0,2019-2020,1,1,Athletic,Barcelona,home,2019-2020,1,6.0,Athletic,...,15.0,Barcelona,1,0.0,1.0,-1.0,0.0,1.0,0.0,0.0
1,2019-2020,1,1,Athletic,Barcelona,home,2019-2020,1,6.0,Athletic,...,9.0,Barcelona,2,5.0,3.0,2.0,1.0,1.0,0.0,3.0
2,2019-2020,1,1,Athletic,Barcelona,home,2019-2020,1,6.0,Athletic,...,8.0,Barcelona,3,7.0,5.0,2.0,1.0,1.0,1.0,4.0
3,2019-2020,1,1,Athletic,Barcelona,home,2019-2020,1,6.0,Athletic,...,5.0,Barcelona,4,12.0,7.0,5.0,2.0,1.0,1.0,7.0
4,2019-2020,1,1,Athletic,Barcelona,home,2019-2020,1,6.0,Athletic,...,8.0,Barcelona,5,12.0,9.0,3.0,2.0,2.0,1.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363683,2019-2020,2,42,Dep. La Coruña,CF Fuenlabrada,home,2019-2020,2,19.0,Dep. La Coruña,...,10.0,CF Fuenlabrada,38,41.0,37.0,4.0,12.0,11.0,15.0,51.0
1363684,2019-2020,2,42,Dep. La Coruña,CF Fuenlabrada,home,2019-2020,2,19.0,Dep. La Coruña,...,9.0,CF Fuenlabrada,39,42.0,37.0,5.0,13.0,11.0,15.0,54.0
1363685,2019-2020,2,42,Dep. La Coruña,CF Fuenlabrada,home,2019-2020,2,19.0,Dep. La Coruña,...,7.0,CF Fuenlabrada,40,43.0,37.0,6.0,14.0,11.0,15.0,57.0
1363686,2019-2020,2,42,Dep. La Coruña,CF Fuenlabrada,home,2019-2020,2,19.0,Dep. La Coruña,...,6.0,CF Fuenlabrada,41,46.0,38.0,8.0,15.0,11.0,15.0,60.0


In [43]:
le = LabelEncoder()
df_test['prediction'] = le.fit_transform(df_test['result'])
df_test["home"] = le.fit_transform(df_test["home_team"])

In [44]:
X_test = df_test[["home",'rank', 'GD', "W", "Pts"]] # he cogido estás aleatoriamente
y_test = df_test['prediction']

In [45]:
# Crear y entrenar un modelo de clasificación (por ejemplo, Random Forest)
model = LogisticRegression()
model.fit(X_train, y_train)

/home/la-quiniela-main/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [46]:
# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

In [47]:
# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy}')

Precisión del modelo: 0.45270325763664415


In [48]:
# Mostrar un informe de clasificación
print(classification_report(y_test, y_pred, target_names=['home_win(1)', 'visitor_win(2)', 'tie(X)']))

                precision    recall  f1-score   support

   home_win(1)       0.45      0.89      0.60    582888
visitor_win(2)       0.42      0.15      0.22    428568
        tie(X)       0.49      0.10      0.16    352232

      accuracy                           0.45   1363688
     macro avg       0.45      0.38      0.33   1363688
  weighted avg       0.45      0.45      0.37   1363688

